# Apply a naive beta cooling

In [ ]:
import numpy as np
from regressor import LinearRegressor

import inversion_ideas as ii

## Synthetic data

In [ ]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

In [ ]:
# Build the X matrix
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [ ]:
# Generate synthetic data with noise
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
std_err =  1e-2 * maxabs
noise = rng.normal(scale=std_err, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

## Naive inversion with beta scheduling

In [ ]:
simulation = LinearRegressor(X)

In [ ]:
uncertainty = std_err * np.ones_like(synthetic_data)
data_misfit = ii.DataMisfit(synthetic_data, uncertainty, simulation)
data_misfit

In [ ]:
smallness = ii.TikhonovZero(n_params)
smallness

In [ ]:
beta_0 = 1e4
regularization = beta_0 * smallness
regularization

In [ ]:
phi = data_misfit + regularization
phi

In [ ]:
minimizer = ii.ConjugateGradient()

In [ ]:
initial_model = np.zeros(n_params)

In [ ]:
model = initial_model.copy()

chi_target = 1.0
beta_cooling_factor = 2.0

while True:
    # Minimize the objective function
    model = minimizer(phi, model) 

    # Stopping criteria: chi factor
    dmisfit = data_misfit(model)
    chi_factor = dmisfit / data_misfit.n_data
    if chi_factor <= chi_target:
        break

    # Cool beta
    regularization.multiplier /= beta_cooling_factor


inverted_model = model.copy()

In [ ]:
inverted_model

In [ ]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)